In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

In [2]:
from senmodel.model.utils import *
from senmodel.metrics.nonlinearity_metrics import *
from senmodel.metrics.edge_finder import *
from senmodel.metrics.train_metrics import *
from senmodel.train.train import *

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
from sklearn.preprocessing import LabelEncoder

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss',
    'hours-per-week', 'native-country', 'income'
]
data = pd.read_csv(url, names=columns, na_values=" ?", skipinitialspace=True)
data = data.dropna()

X = data.drop('income', axis=1)
y = data['income']

for col in X.select_dtypes(include=['object']).columns:
    X[col] = LabelEncoder().fit_transform(X[col])

y = LabelEncoder().fit_transform(y)

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)


URLError: <urlopen error [Errno 61] Connection refused>

In [5]:
class TabularDataset(Dataset):
    def __init__(self, features, targets):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.targets = torch.tensor(targets, dtype=torch.long)

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        return self.features[idx], self.targets[idx]

In [6]:
train_dataset = TabularDataset(X_train, y_train)
val_dataset = TabularDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)

In [12]:
class EnhancedFCN(nn.Module):
    def __init__(self, input_size=14, hidden_size=32):
        super(EnhancedFCN, self).__init__()
        self.fc0 = nn.Linear(input_size, hidden_size)
        self.fc1 = nn.Linear(hidden_size, 2)
        self.act = nn.ReLU()

    def forward(self, x):
        x = self.fc1(self.act(self.fc0(x)))
        return x

In [8]:
import wandb

wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: vanyamironov to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
hyperparams = {
    "num_epochs": 10,
    "metric": AbsGradientEdgeMetric(nn.CrossEntropyLoss()),
    "aggregation_mode": "mean",
    "choose_thresholds": {"fc0": 0.8, "fc1": 0.8},
    "threshold": 0.0005,
    "min_delta_epoch_replace": 32,
    "window_size": 5,
    "lr": 3e-4,
    "delete_after": 2,    
}

name = ", ".join(
    f"{key}: {value.__class__.__name__ if key == 'metric' else value}"
    for key, value in hyperparams.items()
)

name

"num_epochs: 256, metric: AbsGradientEdgeMetric, aggregation_mode: mean, choose_thresholds: {'fc0': 0.8}, replace_layers: ['fc0'], threshold: 0.0005, min_delta_epoch_replace: 32, window_size: 5, lr: 0.0003, delete_after: 2"

In [17]:
dense_model = EnhancedFCN()
sparse_model = convert_dense_to_sparse_network(dense_model, layers=[dense_model.fc0, dense_model.fc1])
wandb.finish()
wandb.init(
    project="self-expanding-nets-titanic",
    name=f"titanic, {name}",
    tags=["complex model", "titanic", "multiclass", hyperparams["metric"].__class__.__name__],
)

acc amount,▇▇▇███████████████████▃▃▃▃▃▃▃▃▁▁▁▁▁▁▁▁▁▁
len_choose,▁█
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
params amount,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▄▄▄▄▄▄▄▄████████
params ratio,▆▇▇▇▇▇▇█▇▇▇▇▇▇▆▆▆▆▇▅█▅▆▇▄▇▅▇▇▇▆▆▆▅▇▅▁▁▄▂
params to replace amount,▂▁▁▁▁▁▁▂▁▁▁▁▁▁▁▂▁▁▁▂▂▁▂▁▂▁▂▂▃▂▂▂▂▂▃█▄▅▄▅
train loss,█▇▇▆▅▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train time,▄▃▂▆▂▆▁▁▃▁▂▂▅▂▁▂█▄▇▅▁▁▂▂▃▇▆▃▃▂▅▃▄▄▂▂▄▅▇▆
val accuracy,▁▄▄▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇█▇███████████████
val loss,█▆▅▅▄▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
acc amount,0.00051


In [18]:
train_sparse_recursive(sparse_model, train_loader, train_loader, val_loader, hyperparams)

100%|██████████| 51/51 [00:00<00:00, 308.16it/s]


Epoch 1/256, Train Loss: 0.7019, Val Loss: 0.6510, Val Accuracy: 0.6542


100%|██████████| 51/51 [00:00<00:00, 326.34it/s]


Epoch 2/256, Train Loss: 0.6130, Val Loss: 0.5802, Val Accuracy: 0.7602


100%|██████████| 51/51 [00:00<00:00, 293.92it/s]


...

Epoch 254/256, Train Loss: 0.3002, Val Loss: 0.3329, Val Accuracy: 0.8455


100%|██████████| 51/51 [00:00<00:00, 188.46it/s]


Epoch 255/256, Train Loss: 0.3007, Val Loss: 0.3316, Val Accuracy: 0.8443


100%|██████████| 51/51 [00:00<00:00, 159.63it/s]


Epoch 256/256, Train Loss: 0.3002, Val Loss: 0.3320, Val Accuracy: 0.8437
